# Chain

In [1]:
import os
import openai
os.environ["OPENAI_API_TYPE"] = "azure"
os.environ["OPENAI_API_VERSION"] = "2023-09-15-preview"

openai.api_type = os.getenv("OPENAI_API_TYPE")
openai.api_base = os.getenv("OPENAI_API_BASE")
openai.api_version = os.getenv("OPENAI_API_VERSION")
openai.api_key = os.getenv("OPENAI_API_KEY")

In [11]:
import pandas as pd
df = pd.read_csv('Data.csv', sep=';')

In [12]:
df.head() 

,Product,Review
0,iPhone,"Great phone, highly recommend!"
1,Macbook,"Excellent laptop, very fast and reliable."
2,AirPods,"Amazing sound quality, love them!"
3,iPad,Perfect for taking notes and browsing the web.
4,Apple Watch,Very convenient for tracking fitness and recei...


## LLM Chain

In [14]:
from langchain.chat_models import AzureChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [15]:
llm = AzureChatOpenAI(
    deployment_name="gpt35old",
    model_name="gpt-35-turbo",
    temperature=0.0,
)

In [16]:
prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe a company that makes {product}?",
)

In [17]:
chain = LLMChain(llm=llm, prompt=prompt)
product = "Apple Watch"
chain.run(product)

'Apple Inc.'

## Simple Sequential Chain

In [18]:
from langchain.chains import SimpleSequentialChain

In [22]:
prompt_1 = ChatPromptTemplate.from_template(
    "What is the best name to describe a company that makes {product}?",
)

chain_1 = LLMChain(llm=llm, prompt=prompt_1)

In [23]:
prompt_2 = ChatPromptTemplate.from_template(
    "Write a 20 words description for the following company:{company_name}",
)

chain_2 = LLMChain(llm=llm, prompt=prompt_2)

In [24]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_1, chain_2], verbose=True)

In [25]:
overall_simple_chain.run(product)



> Entering new SimpleSequentialChain chain...
Apple Inc.
Apple Inc. is a multinational technology company that designs, develops, and sells consumer electronics, computer software, and online services.

> Finished chain.


'Apple Inc. is a multinational technology company that designs, develops, and sells consumer electronics, computer software, and online services.'

## Sequential Chain

In [26]:
from langchain.chains import SequentialChain

In [43]:
# prompt template 1: translate to english
first_prompt = ChatPromptTemplate.from_template(
    "Translate the following review to Chinese:"
    "\n\n{Review}"
)
# chain 1: input= Review and output= English_Review
chain_one = LLMChain(llm=llm, prompt=first_prompt, 
                     output_key="Chinese_Review"
                    )

In [44]:
second_prompt = ChatPromptTemplate.from_template(
    "Can you summarize the following review in 1 word:"
    "\n\n{Chinese_Review}"
)
# chain 2: input= English_Review and output= summary
chain_two = LLMChain(llm=llm, prompt=second_prompt, 
                     output_key="summary"
                    )

In [45]:
# prompt template 3: translate to english
third_prompt = ChatPromptTemplate.from_template(
    "What language is the following review:\n\n{Chinese_Review}"
)
# chain 3: input= Review and output= language
chain_three = LLMChain(llm=llm, prompt=third_prompt,
                       output_key="language"
                      )


In [46]:
# prompt template 4: follow up message
fourth_prompt = ChatPromptTemplate.from_template(
    "Write a follow up response to the following "
    "summary in the specified language:"
    "\n\nSummary: {summary}\n\nLanguage: {language}"
)
# chain 4: input= summary, language and output= followup_message
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key="followup_message"
                     )


In [47]:
# overall_chain: input= Review 
# and output= English_Review,summary, followup_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["Review"],
    output_variables=["Chinese_Review", "summary","followup_message"],
    verbose=True
)

In [51]:
review = df.Review[5]
overall_chain(review)



> Entering new SequentialChain chain...

> Finished chain.


{'Review': 'Great for graphic design and video editing.',
 'Chinese_Review': '非常适合图形设计和视频编辑。',
 'summary': 'Suitable.',
 'followup_message': '回复：合适。\n\n非常感谢您的评论。我们很高兴听到我们的产品对您来说是合适的。我们将继续努力提供高质量的产品和服务，以满足您的需求。如果您有任何其他反馈或建议，请随时与我们联系。再次感谢您的支持！'}

## Router Chain

In [ ]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""